# Data Access through flux

Data must be accessible for it to be useful.

Standard APIs let us bridge the gap between new technologies (Zarr/Icechunk/Arraylake) & legacy workflows.

This means we can make our data fit different formats.

This notebook demonstrates a few ways this can work using *Flux* - Earthmover's Data Delivery service.

Funamentally Flux enables access through standard Web APIs

```python
    "https://compute.earthmover.io"
    # the "web tiles" service
    "/v1/services/tiles/"
    # {org}/{repo}
    "earthmover-public/era5-surface-aws"
    # {branch}/{path/to/group}/{service}
    "/main/spatial/tiles/"
    # {tiling-system}/... <- as required by the OGC Tiles standard
    "WebMercatorQuad/{z}/{y}/{x}"
    # query parameters, here selecting variable and setting colorbar range
    "?variables=tcc&colorscalerange=0,0.0000001"
    # setting size of PNGs
    "&width=512&height=512"
```

Read the docs about Flux here: https://docs.earthmover.io/flux/

_As we work through the material below, notice how we never use the Arraylake Client_

## Grab tabular formats via the EDR API


### Serve data to tabular library users

Here we use pandas and specify the _csv_ format

In [ ]:
import pandas as pd

pd.read_csv(
    "https://compute.earthmover.io/v1/services/edr/"
    # {org}/{repo}
    "earthmover-public/era5-surface-aws"
    # {branch}/{path/to/group}/edr/{type-of-query}?
    "/main/temporal/edr/position?"
    # select variable `sd` at coordinates
    "parameter-name=sd&coords=POINT(286%2040.0150)"
    # output format
    "&f=csv"
)

### Export a csv to a file

In [ ]:
%%bash

curl "https://compute.earthmover.io/v1/services/edr/"\
"earthmover-public/era5-surface-aws"\
"/main/temporal/edr/position?"\
"parameter-name=sd&coords=POINT(286%2040.0150)"\
"&f=csv" > era5_timeseries.csv

In [ ]:
!head era5_timeseries.csv

### Exercise: Arraylake data in Google Sheets

Use the `IMPORTDATA` function to grab data from the csv link.

Grabbing the whole timeseries will fail (too much data for the sheet); but you can specify a datetime range using `&datetime=2020-01-01/2020-12-31`.

Your cell should look like `=IMPORTDATA(URL)`


### Exercise 2: Does this work in Excel?
### Exercise 3: Adapt that solution to your own dataset.

## Serve map tiles via the Tiles service

### In the notebook

In [ ]:
from ipyleaflet import Map, TileLayer, basemaps

url = (
    "https://compute.earthmover.io"
    "/v1/services/tiles/earthmover-public/era5-surface-aws"
    "/main/spatial/tiles/WebMercatorQuad/{z}/{y}/{x}"
    "?variables=t2&colorscalerange=280,310&width=512&height=512"
)
m = Map(center=(-40, 160), zoom=4)
tile_layer = TileLayer(url=url, show_loading=True, opacity=0.5)
m.add(tile_layer)
m

### As a standalone MapBox app

See `mapbox.html` in this same directory. Right click and open in a new browser window.

Note this HTML file uses a token that I will delete after the workshop. Replace `mapboxgl.accessToken` with your token, if you try this in the future.

### Exercise: Update `mapbox.html` to visualize your own data

## Access your data in netCDF tools using OPeNDAP


Let's use `ncdump -h` to get a listing of all metadata in this _Zarr_ group

In [ ]:
!ncdump -h https://compute.earthmover.io/v1/services/dap2/earthmover-public/era5-surface-aws/main/temporal/opendap

### Exercise: work with your data in your favorite netcdf viewer (ncview?)

## Discussion:

1. What other programs or interfaces would you like to expose for your data?
2. How do you see Flux being useful in your work?